In [2]:
import torch

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")

In [66]:
code = """
class AgentMemory:
    def __init__(self):
        self.working_memory = {"current_task": None, "intermediate_steps": []}
        self.episodic_memory = []

    def add_step(self, thought: str, action: str, action_input: dict, observation: str):
        step = {
            "thought": thought,
            "action": action,
            "action_input": action_input,
            "observation": observation
        }
        self.working_memory["intermediate_steps"].append(step)
        self.episodic_memory.append(step)
        print(f"🧠 Memory Updated: Added step for action '{action}'.")

    def get_context(self) -> str:
        return json.dumps(self.working_memory, indent=2)

    def clear_working_memory(self):
        self.working_memory = {"current_task": None, "intermediate_steps": []}


class ToolRegistry:
    def __init__(self):
        self.tools = {
            "get_current_time": self.get_current_time,
            "calculator": self.calculator,
        }
        print(f"🔧 Tools Registered: {list(self.tools.keys())}")

    def get_tool_names(self) -> list[str]:
        return list(self.tools.keys())

    def use_tool(self, tool_name: str, tool_input: dict) -> str:
        if tool_name in self.tools:
            try:
                result = self.tools[tool_name](**tool_input)
                return f"Tool '{tool_name}' returned: {result}"
            except Exception as e:
                return f"Error using tool '{tool_name}': {e}"
        else:
            return f"Error: Tool '{tool_name}' not found."

    def get_current_time(self, timezone: str) -> str:
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    def calculator(self, expression: str) -> str:
        return str(eval(expression, {"__builtins__": {}}, {}))


class HolisticAgent:
    def __init__(self, llm_client, memory, tool_registry):
        self.llm = llm_client
        self.memory = memory
        self.tools = tool_registry
        self.max_steps = 5

    def run(self, user_query: str):
        self.memory.clear_working_memory()
        self.memory.working_memory["current_task"] = user_query

        for i in range(self.max_steps):
            print(f"\n--- STEP {i+1} ---")
            prompt = self._build_prompt(user_query)

            response_str = self.llm.query(prompt)
            try:
                response_json = json.loads(response_str)
            except json.JSONDecodeError:
                print("Error: LLM returned invalid JSON. Stopping.")
                break

            thought = response_json.get("thought", "")
            action = response_json.get("action", "Final Answer")
            action_input = response_json.get("action_input", "")

            if action == "Final Answer":
                print(f"\n✅ Final Answer: {action_input}")
                break
            
            print(f"🤖 Action: Using tool '{action}' with input '{action_input}'")
            observation = self.tools.use_tool(action, action_input)
            print(f"👀 Observation: {observation}")
            
            self.memory.add_step(thought, action, action_input, observation)
        else:
            print("\n⚠️ Agent stopped: Reached maximum steps.")
            
    def _build_prompt(self, query: str) -> str:
        return f\"""
        You are an intelligent agent. Your goal is to solve the user's request by reasoning and using the available tools.
        Follow the Thought-Action-Observation loop (ReAct pattern).

        Available Tools: {self.tools.get_tool_names()}

        User Query: {query}
        
        Previous Steps (Memory):
        {self.memory.get_context()}

        Your response MUST be a JSON object with 'thought', 'action', and 'action_input'.
        'action' should be one of the available tools or 'Final Answer'.
        \"""


if __name__ == "__main__":
    llm_client = MockLLMClient()
    agent_memory = AgentMemory()
    tool_registry = ToolRegistry()
    
    agent = HolisticAgent(
        llm_client=llm_client,
        memory=agent_memory,
        tool_registry=tool_registry
    )

    agent.run("What is the of Sri Lanka, what is the current time in Colombo, hi and what is 25 * 8?")

"""

In [132]:
chunk_size = 128
stride = 68

In [128]:
all_tokens = tokenizer.encode(code, add_special_tokens=False)
total_length = len(all_tokens)

In [ ]:
if total_length <= chunk_size:
    inputs = tokenizer(code,return_tensors='pt')
else:
    chunks = []
    for i in range(0,total_length,stride):
        chunk = all_tokens[i:i+chunk_size]
        chunks.append(chunk)
    input_ids = [tokenizer.build_inputs_with_special_tokens(chunk) for chunk in chunks]
    max_len = max(len(ids) for ids in input_ids)
    attention_masks = []
    padded_input_ids = []
    for chunk in input_ids:
        padding_length = max_len - len(chunk)
        padded_input_ids.append(chunk+[tokenizer.pad_token_id]*padding_length)
        attention_masks.append([1]*len(chunk)+[0]*padding_length)
    inputs = {'input_ids': torch.tensor(padded_input_ids), 'attention_mask': torch.tensor(attention_masks)}


with torch.no_grad():
    outputs = model(**inputs)

In [178]:
code_embedding1 = torch.mean(outputs.pooler_output,0)

In [ ]:
expanded_attention_mask = inputs['attention_mask'].unsqueeze(-1).expand(outputs.last_hidden_state.shape)
masked_embeddings = expanded_attention_mask * outputs.last_hidden_state
code_embedding2 = masked_embeddings.sum(1)/ expanded_attention_mask.sum(1)